In [1]:
import rosbag
import cv2
from cv_bridge import CvBridge
import os
from fnmatch import fnmatch
import tf
import rospy
import rosbag
from tf2_msgs.msg import TFMessage
from std_msgs.msg import Header
from sensor_msgs.msg import CameraInfo, Imu, PointField, NavSatFix
import sensor_msgs.point_cloud2 as pc2
from geometry_msgs.msg import TransformStamped, TwistStamped, Transform
from cv_bridge import CvBridge
import numpy as np
from python_pointclouds import pointclouds

%matplotlib inline

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bag_file_dir = '/media/sf_vol/didi/Didi-Training-Release-1'


* Work through the bag files and show what's in them

In [31]:
files = os.listdir(bag_file_dir)
for file in files:
    if fnmatch(file, '*.bag'):
        print ('Processing bag {}'.format(file))
        bag = rosbag.Bag(os.path.join(bag_file_dir, file))
        info = bag.get_type_and_topic_info()[1]
        topics = info.keys()
        #print ('  Topics: {}'.format(topics))
        #for topic in topics:
        #    print ('Topic {}: {}'.format(topic, info[topic]))
        #print ('*'*20)            
        
        

Processing bag 5mph.bag
  Topics: ['/diagnostics_toplevel_state', '/gps/rtkfix', '/cloud_nodelet/parameter_descriptions', '/obs1/gps/rtkfix', '/gps/time', '/image_raw', '/radar/points', '/rosout', '/obs1/gps/time', '/tf', '/velodyne_points', '/velodyne_packets', '/velodyne_nodelet_manager/bond', '/obs1/gps/fix', '/gps/fix', '/radar/tracks', '/diagnostics', '/cloud_nodelet/parameter_updates', '/radar/range', '/diagnostics_agg']
Processing bag approach_1.bag
  Topics: ['/diagnostics_toplevel_state', '/gps/rtkfix', '/cloud_nodelet/parameter_descriptions', '/obs1/gps/rtkfix', '/gps/time', '/image_raw', '/radar/points', '/rosout', '/obs1/gps/time', '/tf', '/velodyne_points', '/velodyne_packets', '/velodyne_nodelet_manager/bond', '/obs1/gps/fix', '/gps/fix', '/radar/tracks', '/diagnostics', '/cloud_nodelet/parameter_updates', '/radar/range', '/diagnostics_agg']
Processing bag approach_2.bag
  Topics: ['/diagnostics_toplevel_state', '/gps/rtkfix', '/cloud_nodelet/parameter_descriptions', '/ob

Extract the following data:
    
* rtkfix is thh cartesian coordinates of 'things' (what?)
* /gps/rtkfix also includes the capture vehicle
* /obs1/ topic namespace is the obstacle (will be renamed)
* /velodyne_packets and /velodyne points both available (points won't be in later releases!!)



More detailed investigation of a single bag...

*TODO* Turn this into a function


In [7]:
from sensor_msgs.msg import PointCloud2, PointField
import numpy as np
import struct

fmt_full = ''

_DATATYPES = {}
_DATATYPES[PointField.INT8]    = ('b', 1)
_DATATYPES[PointField.UINT8]   = ('B', 1)
_DATATYPES[PointField.INT16]   = ('h', 2)
_DATATYPES[PointField.UINT16]  = ('H', 2)
_DATATYPES[PointField.INT32]   = ('i', 4)
_DATATYPES[PointField.UINT32]  = ('I', 4)
_DATATYPES[PointField.FLOAT32] = ('f', 4)
_DATATYPES[PointField.FLOAT64] = ('d', 8)

_NP_TYPES = {
    np.dtype('uint8')   :   (PointField.UINT8,  1),
    np.dtype('int8')    :   (PointField.INT8,   1),
    np.dtype('uint16')  :   (PointField.UINT16, 2),
    np.dtype('int16')   :   (PointField.INT16,  2),
    np.dtype('uint32')  :   (PointField.UINT32, 4),
    np.dtype('int32')   :   (PointField.INT32,  4),
    np.dtype('float32') :   (PointField.FLOAT32,4),
    np.dtype('float64') :   (PointField.FLOAT64,8)
}

def pointcloud2_to_array(msg):
    global fmt_full
    if not fmt_full:
        fmt = _get_struct_fmt(msg)
        fmt_full = '>' if msg.is_bigendian else '<' + fmt.strip('<>')*msg.width*msg.height
    # import pdb; pdb.set_trace()
    unpacker = struct.Struct(fmt_full)
    unpacked = np.asarray(unpacker.unpack_from(msg.data))
    return unpacked
    # unpacked.reshape(msg.height, msg.width, len(msg.fields))

def _get_struct_fmt(cloud, field_names=None):
    fmt = '>' if cloud.is_bigendian else '<'
    #print 'bigendian={}'.format(fmt)
    offset = 0
    for field in (f for f in sorted(cloud.fields, key=lambda f: f.offset) if field_names is None or f.name in field_names):
        #print field
        if offset < field.offset:
            fmt += 'x' * (field.offset - offset)
            offset = field.offset
        if field.datatype not in _DATATYPES:
            print >> sys.stderr, 'Skipping unknown PointField datatype [%d]' % field.datatype
        else:
            datatype_fmt, datatype_length = _DATATYPES[field.datatype]
            fmt    += field.count * datatype_fmt
            offset += field.count * datatype_length

    #print 'return fmt={}'.format(fmt)
    return fmt

In [3]:
bag = rosbag.Bag(os.path.join(bag_file_dir, 'overtake.bag'))
cvbridge = CvBridge()
count = 0
gps_count = 0
images = []
point_clouds = []
_gps_fix = []
_obs1_gps_fix = []

for topic, msg, t in bag.read_messages(
    topics = [
        #'/gps/time',
        '/gps/fix',
        '/gps/rtkfix',
        '/obs1/gps/fix',
        '/image_raw',
        '/velodyne_points' #,
        #'/velodyne_packets',
        #'/radar/range',
        #'/radar/points',
        #'/radar/tracks'
    ]):
    print ('topic: {}'.format(topic))
    if topic == '/image_raw':
        img = cvbridge.imgmsg_to_cv2(msg, "bgr8")
        cv2.imshow('raw image',img)
        key = cv2.waitKey(1)
    elif topic == '/velodyne_points':
        #print ('*'*50)
        #print (msg)
        #title = 'points'
        #img = np.array([x,z]
        point_clouds.append(msg)
        images.append(img) # Get the image closest to the point cloud we have (testing this works!)
        count +=1 
        
        lidar = pc2.read_points(msg)
        lidar = np.array(list(lidar))
        break
    elif topic == '/obs1/gps/fix':
        #print ('/obs1/gps/fix')
        #print (msg)
        #print ('*'*20)
        latitude = msg.latitude
        longitude = msg.longitude
        altitude = msg.altitude
        _obs1_gps_fix.append((latitude, longitude, altitude))
        #gps_count +=1
    elif topic == '/gps/fix':
        #print ('/gps/fix')
        #print (msg)
        #print ('*'*20)
        latitude = msg.latitude
        longitude = msg.longitude
        altitude = msg.altitude
        _gps_fix.append((latitude, longitude, altitude))
        #gps_count +=1
    #else:
        #print ('topic: {}'.format(topic))
    #if gps_count == 5:
    #    break


print "Processed {} messages".format(count)

gps_fix = np.array(_gps_fix) #.transpose()
obs1_gps_fix = np.array(_obs1_gps_fix) #.transpose()

    

topic: /image_raw
topic: /gps/rtkfix
topic: /image_raw
topic: /velodyne_points
Processed 1 messages


In [4]:
print len(point_clouds)
print len(images)
print gps_fix.shape
print obs1_gps_fix.shape

1
1
(0,)
(0,)


In [5]:
print msg.header.seq
print msg.header.stamp.secs
print msg.header


10909
1490150408
seq: 10909
stamp: 
  secs: 1490150408
  nsecs: 233622000
frame_id: velodyne


#### LIDAR rendering
From http://ronny.rest/blog/post_2017_03_30_ros3_and_lidar/

In [ ]:
import mayavi.mlab
fig = mayavi.mlab.figure(bgcolor=(0, 0, 0), size=(500, 300))

mayavi.mlab.points3d(lidar[:,0], lidar[:,1], lidar[:,2],
                     lidar[:,0]**2 + lidar[:,1]**2, # distance values for color
                     mode="point",
                     colormap='spectral',
                     figure=fig,
                     )
mayavi.mlab.show()

In [28]:
cv2.destroyAllWindows()

In [11]:
 
def get_cloud_points(cloud):
    pt_x=[]
    pt_y=[]
    pt_z=[]

    for point in pc2.read_points(cloud, skip_nans=True):
        pt_x.append(point[0])
        pt_y.append(point[1])
        pt_z.append(point[2])
        #print 'x:{}, y:{}, z:{}'.format(pt_x, pt_y, pt_z)

    return pt_x, pt_y, pt_z


In [12]:
from cStringIO import StringIO

def plot_point_cloud(cloud):
    mpl.rcParams['legend.fontsize'] = 10
    
    fig = plt.figure()
    #ax = fig.gca(projection='3d')

    pt_x, pt_y, pt_z = get_cloud_points(cloud)
    #ax.scatter(pt_x, pt_y, pt_z, label='points')
    #ax.legend()
    #plt.show()
    
    plt.plot(pt_x, pt_z, 'ro')    
    buffer_ = StringIO()
    fig.savefig(buffer_, format = "png")
    buffer_.seek(0)
    img_array = np.asarray(bytearray(buffer_.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, 1)   # 1 = colour
    plt.close(fig)
    #cv2.imshow('points', img)
    #cv2.waitKey(1)
    
    #plt.show()
    
    #plt.plot(pt_x, pt_y, 'ro')
    #plt.show()
    
    return img

In [13]:
cv2.destroyAllWindows()

In [14]:
for idx in range(0, len(images), 20): 
    print 'Index: {}'.format(idx)
    cv2.imshow('image',images[idx])
    cv2.imshow('points',plot_point_cloud(point_clouds[idx]))
    cv2.waitKey(1)

Index: 0
Index: 20
Index: 40
Index: 60
Index: 80
Index: 100
Index: 120
Index: 140
Index: 160
Index: 180
Index: 200
Index: 220
Index: 240
Index: 260
Index: 280
Index: 300
Index: 320
Index: 340
Index: 360
Index: 380
Index: 400
Index: 420
Index: 440
Index: 460
Index: 480
Index: 500
Index: 520
Index: 540
Index: 560
Index: 580
Index: 600
Index: 620
Index: 640
Index: 660
Index: 680
Index: 700
Index: 720
Index: 740
Index: 760
Index: 780
Index: 800
Index: 820
Index: 840
Index: 860
Index: 880
Index: 900
Index: 920
Index: 940
Index: 960
Index: 980
Index: 1000
Index: 1020
Index: 1040
Index: 1060
Index: 1080
Index: 1100
Index: 1120
Index: 1140
Index: 1160
Index: 1180
Index: 1200
Index: 1220
Index: 1240
Index: 1260
Index: 1280
Index: 1300
Index: 1320
Index: 1340
Index: 1360
Index: 1380
Index: 1400
Index: 1420
Index: 1440
Index: 1460
Index: 1480
Index: 1500
Index: 1520
Index: 1540
Index: 1560
Index: 1580
Index: 1600
Index: 1620
Index: 1640
Index: 1660
Index: 1680
Index: 1700
Index: 1720
Index: 1740


In [15]:
cv2.destroyAllWindows()

In [16]:
print msg.header.seq
print msg.header.stamp.secs

9064
1490155973
